# Load model from google drive

In [53]:
from google.colab import drive
drive.mount('/content/gdrive')


#set your path in google drive where model is kept
PATH = '/content/gdrive/MyDrive/ResNet/resnet101.135.h5';

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Load model

In [55]:
import torch
import torchvision
import torch.nn as nn
from torchvision.models import resnet101

model = resnet101(pretrained=False, num_classes=10)

model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()


model.load_state_dict(torch.load(PATH))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

In [56]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(model)

pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Total Params: ", pytorch_total_params)

+------------------------------+------------+
|           Modules            | Parameters |
+------------------------------+------------+
|         conv1.weight         |    1728    |
|          bn1.weight          |     64     |
|           bn1.bias           |     64     |
|    layer1.0.conv1.weight     |    4096    |
|     layer1.0.bn1.weight      |     64     |
|      layer1.0.bn1.bias       |     64     |
|    layer1.0.conv2.weight     |   36864    |
|     layer1.0.bn2.weight      |     64     |
|      layer1.0.bn2.bias       |     64     |
|    layer1.0.conv3.weight     |   16384    |
|     layer1.0.bn3.weight      |    256     |
|      layer1.0.bn3.bias       |    256     |
| layer1.0.downsample.0.weight |   16384    |
| layer1.0.downsample.1.weight |    256     |
|  layer1.0.downsample.1.bias  |    256     |
|    layer1.1.conv1.weight     |   16384    |
|     layer1.1.bn1.weight      |     64     |
|      layer1.1.bn1.bias       |     64     |
|    layer1.1.conv2.weight     |  

## Number of FLOPS

In [63]:
pip install -U fvcore

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
from fvcore.nn import FlopCountAnalysis
from fvcore.nn import flop_count_table

input =  torch.rand(1, 3, 32, 32)

flops = FlopCountAnalysis(model, input)
print(flop_count_table(flops))
print("Total number of FLOPS: ", flops.total())

| module                 | #parameters or shape   | #flops     |
|:-----------------------|:-----------------------|:-----------|
| model                  | 42.513M                | 2.52G      |
|  conv1                 |  1.728K                |  1.769M    |
|   conv1.weight         |   (64, 3, 3, 3)        |            |
|  bn1                   |  0.128K                |  0.131M    |
|   bn1.weight           |   (64,)                |            |
|   bn1.bias             |   (64,)                |            |
|  layer1                |  0.216M                |  0.221G    |
|   layer1.0             |   75.008K              |   76.808M  |
|    layer1.0.conv1      |    4.096K              |    4.194M  |
|    layer1.0.bn1        |    0.128K              |    0.131M  |
|    layer1.0.conv2      |    36.864K             |    37.749M |
|    layer1.0.bn2        |    0.128K              |    0.131M  |
|    layer1.0.conv3      |    16.384K             |    16.777M |
|    layer1.0.bn3        